In [2]:
from helpers import load_CSD_data, make_element_wise_environments
import numpy as np
from rascal.representations import SphericalInvariants
from rascal.models.kernels import Kernel
from sklearn.kernel_ridge import KernelRidge
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species
from sklearn.metrics import mean_squared_error, mean_absolute_error


PATH_TRAIN = "CSD-2k_relaxed_shifts.txt"
PATH_TEST = "CSD-500.txt"
structures_test, shifts_test = load_CSD_data(PATH_TEST,random_subsample=10,prop_string="cs_iso")
structures_train, shifts_train = load_CSD_data(PATH_TRAIN,random_subsample=10,prop_string="cs_iso")

FileNotFoundError: [Errno 2] No such file or directory: 'CSD-500.txt'

In [2]:
hypers = {"soap_type": "PowerSpectrum",
          "interaction_cutoff": 3,
          "radial_basis": "GTO",
          "max_radial": 9,
          "max_angular": 9,
          "gaussian_sigma_constant": 0.3,
          "gaussian_sigma_type":"Constant",
          "cutoff_function_type":"ShiftedCosine",
          "cutoff_smooth_width": 0.5,
          "normalize": True,
          "compute_gradients":False,
          "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4),
          #"optimization": dict(Spline=dict(accuracy=1.0e-05))
          "expansion_by_species_method":'user defined',
          "global_species":[1,6,7,8]
         }

In [20]:
for structure in structures_test: 
    mask_center_atoms_by_species(structure,species_select=[1])
for structure in structures_train: 
    mask_center_atoms_by_species(structure,species_select=[1])

H_train_shifts = np.array([shift for structure in structures_train for shift in structure.arrays["CS"][structure.arrays["center_atoms_mask"]]])        
H_test_shifts = np.array([shift[0] for structure in structures_test for shift in structure.arrays["CS"][structure.arrays["center_atoms_mask"]]]) 
H_test_shifts_ML = np.array([shift[1] for structure in structures_test for shift in structure.arrays["CS"][structure.arrays["center_atoms_mask"]]]) 

In [4]:

calculator = SphericalInvariants(**hypers)
atoms_list_train = calculator.transform(structures_train)

In [5]:
X_train = calculator.transform(structures_train).get_features(calculator)
X_test = calculator.transform(structures_test).get_features(calculator)

In [32]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics.pairwise import polynomial_kernel

import numpy as np

clf = KernelRidge(alpha=1e-02,kernel='poly', degree=2, coef0=0,gamma=1.)
clf.fit(X_train, H_train_shifts)

KernelRidge(alpha=0.01, coef0=0, degree=2, gamma=1.0, kernel='poly')

In [34]:
predictions = clf.predict(X_test)

In [35]:
mean_squared_error(H_test_shifts,predictions,squared=False)

0.882802633457133

In [ ]:
def build_multiscale_kernel(structures,hypers,kernel_weights=[256,128,32,8,8,1],kernel_distances=[2,3,4,5,6,7]):
    sum_weights = 
    msk_kernel = 
    for distance, weight in zip(kernel_distances,kernel_weights):
        hypers["interaction_cutoff"] = distance
        calculator = SphericalInvariants(**hypers)
        atoms_list_train = calculator.transform(structures_train)
        X_train = calculator.transform(structures_train).get_features(calculator)
        msk_tmp = sklearn.metrics.pairwise.polynomial_kernel(X_train, degree=2, gamma=1., coef0=0)
        msk_kernel += weight * 
        
    msk_kernel = 1/sum_weights * kernel    
    return msk_kerneld